# Import

In [ ]:
import sys 
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, recall_score, precision_score, f1_score, accuracy_score

In [ ]:
# VARS 

PATH = '.\\data\\' if sys.platform == 'win32' else './data/'
SOLUTIONS = '.\\submissions\\' if sys.platform == 'win32' else './submissions/'

In [ ]:
train = pd.read_csv(
    PATH + 'train_dataset_train.csv',
    sep=';',
    dtype={
        'PATIENT_SEX':str, 
        'MKB_CODE':str, 
        'ADRES':str, 
        'VISIT_MONTH_YEAR':str, 
        'AGE_CATEGORY':str, 
        'PATIENT_ID_COUNT':int}
    )

test = pd.read_csv(
    PATH+'test_dataset_test.csv',
    sep=';', 
    dtype={
        'PATIENT_SEX':str,
        'MKB_CODE':str,
        'ADRES':str,
        'VISIT_MONTH_YEAR':str,
        'AGE_CATEGORY':str}
    )
orig_test = test.copy()
orig_train = train.copy()

# Baseline №1 - Простая аггрегация 

Предложенный Михаилом Марьиным в чате способ на простой аггрегации и среднем. Выдает значение **0.66** на **публичном лидерборде**

In [ ]:
test.merge(train[train['VISIT_MONTH_YEAR'].isin(['01.22', '02.22', '03.22'])].groupby(['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'], as_index=False)['PATIENT_ID_COUNT'].mean(),
    on=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'],
    how='left').fillna(1).astype({'PATIENT_ID_COUNT': int}
    ).to_csv(SOLUTIONS + 'baseline_mm.csv', sep=';', index=None)

Попробуем убрать из этого датасета непопулярные коды мкб 

In [ ]:
agg_df = train[train['VISIT_MONTH_YEAR'].isin(['01.22', '02.22', '03.22'])]
agg_func_math = ['sum', 'count']
mkb_info_df = agg_df.groupby('MKB_CODE')['PATIENT_ID_COUNT'].agg(agg_func_math).round(2).sort_values(by='sum', ascending=False)
unpopular = mkb_info_df[mkb_info_df['count'] == 1].index.tolist()
agg_df = agg_df.query(f'MKB_CODE not in {unpopular}')

In [ ]:
test.merge(agg_df.groupby(['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'], as_index=False)['PATIENT_ID_COUNT'].mean(),
    on=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'],
    how='left').fillna(1).astype({'PATIENT_ID_COUNT': int}
    ).to_csv(SOLUTIONS + 'baseline_threshold.csv', sep=';', index=None)

Не влияет на итоговый скор на паблике. Будем пытаться бить 0,66 на кросс-валидации 

# CatBoostClassifier, пытаемся предсказать Target_Range на полном датасете

In [ ]:
train_df = pd.read_csv(
    PATH + 'prepared_df.csv',
    index_col=0,
    dtype={
        'PATIENT_SEX':str, 
        'MKB_CODE':str, 
        'ADRES':str, 
        'VISIT_MONTH_YEAR':str, 
        'AGE_CATEGORY':str, 
        'PATIENT_ID_COUNT':int}
        )

In [ ]:
train_df = train_df.sort_values(by='DATE').reset_index(drop=True)
train_df.head(5)

In [ ]:
tscv = TimeSeriesSplit()
classifier = CatBoostClassifier(
    task_type='GPU', 
    random_seed=42,
)
regressor = CatBoostRegressor(
    task_type='GPU', 
    random_seed=42,
)

Напишем функции для обучения модели и кросс-валидации 

In [ ]:
def make_a_regression(df, tscv, model, target, columns_to_drop, cat_features):
    columns_to_drop.append(target)
    fold = 0
    for train_index, test_index in tscv.split(df):
        fold += 1 
        X_train = df.drop(columns_to_drop, axis=1).iloc[train_index]
        X_test = df.drop(columns_to_drop, axis=1).iloc[test_index]
        y_train = df[target].iloc[train_index]
        y_test = df[target].iloc[test_index]

        pool_train = Pool(X_train, y_train, cat_features=cat_features)
        pool_test = Pool(X_test, cat_features=cat_features)

        model.fit(pool_train, silent=True, plot=True)
        y_pred = model.predict(pool_test)
        y_pred = [1 if round(value) <= 0 else round(value) for value in y_pred]
        
        print(f'fold {fold}')
        print('R2: ', r2_score(y_test, y_pred))
        print('#'*50)
    
    return model

In [ ]:
def make_a_classification(df, tscv, model, target, columns_to_drop, cat_features, average=None):
    columns_to_drop.append(target)
    fold = 0
    for train_index, test_index in tscv.split(df):
        fold += 1 
        X_train = df.drop(columns_to_drop, axis=1).iloc[train_index]
        X_test = df.drop(columns_to_drop, axis=1).iloc[test_index]
        y_train = df[target].iloc[train_index]
        y_test = df[target].iloc[test_index]

        pool_train = Pool(X_train, y_train, cat_features=cat_features)
        pool_test = Pool(X_test, cat_features=cat_features)

        model.fit(pool_train, silent=True)
        y_pred = model.predict(pool_test)
        
        print(f'fold {fold}')
        print('Accuracy: ',accuracy_score(y_test, y_pred))
        print('Recall: ', recall_score(y_test, y_pred, average=average, zero_division=0)) 
        print('Precision: ', precision_score(y_test, y_pred, average=average, zero_division=0))
        print('F1 score: ', f1_score(y_test, y_pred, average=average, zero_division=0))
        print('#'*50)
        
    return model
    

In [ ]:
cls = make_a_classification(
    df=train_df,
    tscv=tscv,
    model=classifier,
    target='TARGET_RANGE',
    columns_to_drop=['MONTH', 'YEAR', 'CHAPTER', 'PATIENT_ID_COUNT', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID'],
)

Как и ожидалось, модель лучше всего предсказывает значения в пределах 1-10 и плохо справляется со значениями в других пределах, что вызвано в том числе и общей несбалансированностью в датасете и во времени. 

Попробуем оттрешхолдить непопулярные МКБ и посмотреть что получится 

In [ ]:
thr_df = train_df.query(f'MKB_CODE not in {unpopular}') 

In [ ]:
thr_df.TARGET_RANGE.value_counts()

In [ ]:
cls =make_a_classification(
    df=thr_df,
    tscv=tscv,
    model=classifier,
    target='TARGET_RANGE',
    columns_to_drop=['MONTH', 'YEAR', 'CHAPTER', 'PATIENT_ID_COUNT', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID'],
)

Нет существенного влияния. Попробуем убрать значения 1-10 и посмотреть как модель справляется в этом случае. 

In [ ]:
lil_df = train_df[train_df.TARGET_RANGE != '1-10']
lil_df.head()

In [ ]:
lil_df.TARGET_RANGE.value_counts()

In [ ]:
cls =make_a_classification(
    df=lil_df,
    tscv=tscv,
    model=classifier,
    target='TARGET_RANGE',
    columns_to_drop=['MONTH', 'YEAR', 'CHAPTER', 'PATIENT_ID_COUNT', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID'],
)

Такая модель в свою очередь хорошо справляется с предсказанием периода 10-100. Попробуем научить модель бинарной классификации и решить задачу по тому .входит ли значение в период 1-10 или нет. Для этого нужно будет создать дополнительный признак

In [ ]:
train_df['IS_1-10'] = [True if range == '1-10' else False for range in train_df.TARGET_RANGE]

In [ ]:
train_df.head()

Точность модели должна быть не ниже простой угадайки, то есть не ниже: 

In [ ]:
train_df['IS_1-10'].value_counts(normalize=True)

In [ ]:
cls =make_a_classification(
    df=train_df,
    tscv=tscv,
    model=classifier,
    target='IS_1-10',
    columns_to_drop=['TARGET_RANGE', 'MONTH', 'YEAR', 'CHAPTER', 'PATIENT_ID_COUNT', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID'],
)

На всех фолдах модель показывает скор лучше простого угадывания. Возможно стоит попробовать предсказывать нахождение таргета в промежутке 1-10, а после уже уточнять это значение. 

In [ ]:
train_df.drop(['IS_1-10', 'TARGET_RANGE', 'MONTH', 'YEAR', 'CHAPTER', 'PATIENT_ID_COUNT', 'DATE'], axis=1).columns

In [ ]:
cls.get_feature_importance()

Согласно модели наибольшее значение имеют фичи: адрес, возраст и код МКБ соответственно. Запомним сделанные выводы и попробуем реализовать простую регрессию на датасете с кросс-валидацией по времени

# CatBoostRegressor. Кросс-валидация по времени

In [ ]:
train_df = pd.read_csv(
    PATH + 'prepared_df.csv',
    index_col=0,
    dtype={
        'PATIENT_SEX':str, 
        'MKB_CODE':str, 
        'ADRES':str, 
        'VISIT_MONTH_YEAR':str, 
        'AGE_CATEGORY':str, 
        'PATIENT_ID_COUNT':int}
        )

In [ ]:
train_df = train_df.sort_values(by='DATE').reset_index(drop=True)
train_df.head(5)

In [ ]:
tscv = TimeSeriesSplit()

Попробуем модель из коробки. 

In [ ]:
reg = make_a_regression(
    df=train_df,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE', 'MONTH', 'YEAR', 'CHAPTER', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID'],
)

Видно, что значения R2 падают на последних кусках датасета, что, скорее всего, связано с тем, что именно эти периоды выпадают на ковид. Попробуем оставить в трейне год и месяц для обучения 

In [ ]:
reg = make_a_regression(
    df=train_df,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'IS_COVID', 'MONTH', 'YEAR'],
)

Особого эффекта не возымело. Попробуем поучиться только на данных ковидного периода

In [ ]:
covid_df = train_df[train_df.IS_COVID == True]
covid_df.head()

In [ ]:
reg = make_a_regression(
    df=covid_df,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER', 'DATE', 'IS_COVID'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'MONTH', 'YEAR'],
)

Попробуем оттрешхолдить коды МКБ и поучить пару моделей

In [ ]:
agg_func_math = ['sum', 'count']
mkb_info_df = train_df.groupby('MKB_CODE')['PATIENT_ID_COUNT'].agg(agg_func_math).round(2).sort_values(by='sum', ascending=False)
unpopular = mkb_info_df[mkb_info_df['count'] < 10].index.tolist()
thr_df = train_df.query(f'MKB_CODE not in {unpopular}')

In [ ]:
reg = make_a_regression(
    df=thr_df,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'MONTH', 'YEAR', 'IS_COVID'],
)

Попробуем нафитить регрессию на промежутке 1-10

In [ ]:
df110 = train_df[train_df.TARGET_RANGE == '1-10']
df110.head(5)

In [ ]:
reg = make_a_regression(
    df=df110,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER', 'DATE'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'MONTH', 'YEAR', 'IS_COVID'],
)

Видно, что даже в такой задаче Регрессор не может адекватно воспринять и предсказать значения. Попробуем дату как фичу без разбиения на месяц и год

In [ ]:
reg = make_a_regression(
    df=thr_df,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER', 'MONTH', 'YEAR'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY', 'DATE','IS_COVID'],
)

Попробуем зафитить только на 1000 популярных мкб кодов 

In [ ]:
popular = train_df.MKB_CODE.value_counts()[:1000].index.tolist()
thr_df_pop = train_df.query(f'MKB_CODE in {popular}')

In [ ]:
reg = make_a_regression(
    df=thr_df_pop,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['TARGET_RANGE' 'CHAPTER'],
    cat_features=['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY','DATE', 'MONTH', 'YEAR', 'IS_COVID'],
)

In [ ]:
df_2022 = train_df[train_df.YEAR == 2022]
tscv_2022 = TimeSeriesSplit(n_splits=3)

In [ ]:
reg = make_a_regression(
    df=df_2022,
    tscv=tscv,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['PATIENT_SEX', 'TARGET_RANGE', 'CHAPTER', 'DATE', 'MONTH', 'YEAR', 'IS_COVID'],
    cat_features=['MKB_CODE', 'ADRES', 'AGE_CATEGORY'],
)

2020 год и 1-10

In [ ]:
df_2022_1_10 = train_df[(train_df.YEAR == 2022) & (train_df.TARGET_RANGE == '1-10')]
tscv_2022 = TimeSeriesSplit(n_splits=3)

In [ ]:
reg = make_a_regression(
    df=df_2022_1_10,
    tscv=tscv_2022,
    model=regressor,
    target='PATIENT_ID_COUNT',
    columns_to_drop=['PATIENT_SEX', 'TARGET_RANGE', 'CHAPTER', 'DATE', 'MONTH', 'YEAR', 'IS_COVID'],
    cat_features=['MKB_CODE', 'ADRES', 'AGE_CATEGORY'],
)

# Make a submission

In [ ]:
test.PATIENT_SEX = test.PATIENT_SEX.map({'1':'Male', '0':'Female'})
test['AGE_CATEGORY'] = test['AGE_CATEGORY'].map({
    'children': '0-18',
    'young': '18-44',
    'middleage': '45-59',
    'elderly': '60-74', 
    'old': '75-90',
    'centenarians': '90+'
})
# TODO FIX 
test['IS_COVID'] = [True] * test.shape[0]

test['DAY'] = [1] * test.shape[0]
test['MONTH'] = [value[0] for value in test.VISIT_MONTH_YEAR.astype(str).apply(lambda x: x.split('.'))]
test['MONTH'] = test['MONTH'].astype('int64')
test['YEAR'] = ['20' + value[1] if len(value[1]) > 1 else '20' + value[1] + '0' for value in test.VISIT_MONTH_YEAR.astype(str).apply(lambda x: x.split('.'))]
test['YEAR'] = test['YEAR'].astype('int64')
test['DATE'] = pd.to_datetime(test[['DAY', 'MONTH', 'YEAR']])
test = test.drop('DAY', axis=1)

In [ ]:
test.head()

In [ ]:
test.DATE = test.DATE.astype(str)

In [ ]:
pool_test = Pool(test.drop(['VISIT_MONTH_YEAR','PATIENT_SEX','DATE', 'MONTH', 'YEAR', 'IS_COVID'], axis=1), cat_features = ['AGE_CATEGORY', 'MKB_CODE', 'ADRES'])

In [ ]:
solution_prediction = model.predict(pool_test)
solution_prediction = [1 if int(value) <= 0 else int(value) for value in solution_prediction]

In [ ]:
orig_test['PATIENT_ID_COUNT'] = solution_prediction

In [ ]:
orig_test

In [ ]:
orig_test.to_csv(SOLUTIONS + 'solution_23_08_22_2.csv', sep=';', index=None)